# Data Stream Processing - Project 3 : Ideas for Extension (20%)

In [2]:
# Import librairies.
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import groupby


## The data

In [3]:
# Get files
DATAPATH =  "../data/custom_no_streaming_8/folder_2"
files = glob.glob(rf"{DATAPATH}/*")
files

['../data/custom_no_streaming_8/folder_2\\1_3.csv',
 '../data/custom_no_streaming_8/folder_2\\1_4.csv',
 '../data/custom_no_streaming_8/folder_2\\2_3.csv',
 '../data/custom_no_streaming_8/folder_2\\2_4.csv',
 '../data/custom_no_streaming_8/folder_2\\3_3.csv',
 '../data/custom_no_streaming_8/folder_2\\3_4.csv']

In [4]:
# Read anomaly files and concatenate dataframes
dfs = []
for file in files[:-1]:
    df = pd.read_csv(file)
    # Extract the filename without the parent folder path and extension
    filename = file.split('\\')[-1].split('.')[0]
    # Add a new column "anomaly_type" with the extracted filename
    df.insert(1, column="trace_id", value=filename)
    df.rename({"Unnamed: 0": "time"}, axis=1, inplace=True)
    dfs.append(df)

# Concatenate all dataframes except labels.csv
anomaly_df = pd.concat(dfs)
anomaly_df

,time,trace_id,driver_BlockManager_memory_memUsed_MB_value,driver_jvm_heap_used_value,avg_jvm_heap_used_value,avg_executor_filesystem_hdfs_write_ops_value_1_diff,avg_executor_cpuTime_count_1_diff,avg_executor_runTime_count_1_diff,avg_executor_shuffleRecordsRead_count_1_diff,avg_executor_shuffleRecordsWritten_count_1_diff
0,0,1_3,505.0,802632640.0,2.948889e+09,0.0,0.0,0.0,0.0,0.0
1,1,1_3,520.0,811308500.0,2.972305e+09,0.0,0.0,0.0,0.0,0.0
2,2,1_3,536.0,792553660.0,2.652633e+09,0.0,355610140.0,373.0,0.0,9981.0
3,3,1_3,482.0,779442200.0,2.686896e+09,0.0,0.0,0.0,0.0,0.0
4,4,1_3,497.0,789423170.0,2.706626e+09,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
21759,21759,3_3,10447.0,522127260.0,7.867158e+09,12.0,398201800.0,656.0,27356.5,4216.5
21760,21760,3_3,8684.0,554331460.0,8.139178e+09,0.0,0.0,0.0,0.0,0.0
21761,21761,3_3,8882.0,566017860.0,8.361957e+09,0.0,0.0,0.0,0.0,0.0
21762,21762,3_3,9088.0,573878800.0,8.542828e+09,0.0,0.0,0.0,0.0,0.0


In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Charger les données
# Assurez-vous que votre fichier CSV est correctement formaté et accessible
df = anomaly_df.copy()

# Sélectionner les colonnes numériques pertinentes (exclure 'time' et 'trace_id')
cols = df.columns[2:]

# Normaliser les données
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[cols])

# Reshape data pour LSTM [samples, time steps, features]
timesteps = 1  # Vous pouvez ajuster cela selon votre séquence de temps
features = len(cols)
data = df_scaled.reshape(df_scaled.shape[0], timesteps, features)

# Construction de l'autoencodeur LSTM
inputs = Input(shape=(timesteps, features))
encoded = LSTM(10, return_sequences=False)(inputs)
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(features, return_sequences=True)(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Entraîner le modèle
autoencoder.fit(data, data, epochs=50, batch_size=128, validation_split=0.2, shuffle=True)

# Prédire les données reconstruites
predictions = autoencoder.predict(data)

# Calculer l'erreur de reconstruction
mse = np.mean(np.power(data - predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse[:,0], 'True_class': df['label']})

# Identifier les anomalies (Vous pouvez définir un seuil d'erreur)
threshold = np.percentile(error_df.Reconstruction_error.values, 95)
anomalies = error_df[error_df.Reconstruction_error > threshold]




Epoch 1/50

643/643 [==============================] - 6s 5ms/step - loss: 0.6078 - val_loss: 1.4853
Epoch 2/50
643/643 [==============================] - 3s 4ms/step - loss: 0.4735 - val_loss: 1.1849
Epoch 3/50
643/643 [==============================] - 3s 5ms/step - loss: 0.4471 - val_loss: 0.9981
Epoch 4/50
643/643 [==============================] - 3s 5ms/step - loss: 0.4374 - val_loss: 0.8396
Epoch 5/50
643/643 [==============================] - 3s 5ms/step - loss: 0.4322 - val_loss: 0.7745
Epoch 6/50
643/643 [==============================] - 3s 5ms/step - loss: 0.4298 - val_loss: 0.7419
Epoch 7/50
643/643 [==============================] - 2s 4ms/step - loss: 0.4283 - val_loss: 0.7187
Epoch 8/50
643/643 [==============================] - 2s 4ms/step - loss: 0.4272 - val_loss: 0.7095
Epoch 9/50
643/643 [==============================] - 3s 4ms/step - loss: 0.4264 - val_loss: 0.7043
Epoch 10/50
643/643 [==============================] - 2s 4ms/step - loss: 0.4258 - val_loss: 0.7

KeyError: 'label'